## Creating a dataset that is filtered like prescription yield but has a CO2 column and no yield column

In [2]:
import pandas as pd

In [9]:
df = pd.read_csv('/mnt/Datasets/merged_df_wet_and_dry_3_20_23.csv').drop(columns = ['Unnamed: 0'])
df['dry_wet_ratio'] = df['Enter Dry Finished Yield Mass (kg)'] / df['Enter Total Wet Mass Harvested (kg)']
df_season = pd.read_excel('/mnt/Datasets/PC003_consolidated_new (1).xlsx')
df_season['Field Name'] = df_season['Contrato legal'].str[6:]
df_season = df_season[['Field Name', 'Safra']]
df_season = df_season.rename(columns = {'Safra': 'Season', 'Field Name': 'Field_Name'})
df_season = df_season.drop_duplicates().reset_index(drop = True)
df = df.merge(df_season, on = 'Field_Name', how = 'left')
df_combined_parents = df
df_combined_parents['Total_N'] = df_combined_parents['N_as_Urea_kg_per_Ha'] + df_combined_parents['N_as_Nitrate_kg_per_Ha'] + df_combined_parents['N_as_Ammonium_kg_per_Ha']
df_combined_parents['Total_P'] = df_combined_parents['P_as_Phosphorus_kg_per_Ha'] + df_combined_parents['P_as_P2O5_kg_per_Ha']
df_combined_parents['Total_K'] = df_combined_parents['K_as_Potassium_kg_per_Ha'] + df_combined_parents['K_as_K2O_kg_per_Ha']
df_combined_parents = df_combined_parents.drop(columns = ['N_as_Urea_kg_per_Ha', 'N_as_Nitrate_kg_per_Ha', 'N_as_Ammonium_kg_per_Ha', \
                                                          'P_as_Phosphorus_kg_per_Ha', 'P_as_P2O5_kg_per_Ha'])

#Winter
df_winter = df[df['Season'] == 'WINTER'].reset_index(drop = True)[['Field_Name', 'Crop_Protection_Application_Doses', 'Soil_Organic_Matter', 'Yield_kg_per_Ha', \
                                                                  'N_as_Urea_kg_per_Ha', 'N_as_Nitrate_kg_per_Ha', 'N_as_Ammonium_kg_per_Ha', 'P_as_Phosphorus_kg_per_Ha', \
                                                                   'P_as_P2O5_kg_per_Ha', 'K_as_Potassium_kg_per_Ha', 'K_as_K2O_kg_per_Ha', 'dry_wet_ratio']]
df_winter_combined_parents = df_combined_parents[df_combined_parents['Season'] == 'WINTER'].reset_index(drop = True)[['Field_Name', 'Crop_Protection_Application_Doses', 'Soil_Organic_Matter', 'Yield_kg_per_Ha', \
                                                                  'Total_N', 'Total_P', 'Total_K', 'dry_wet_ratio']]
df_dy_wy_ratio_winter  = df_winter[(df_winter['dry_wet_ratio'] > .2) & (df_winter['dry_wet_ratio'] < .6)].reset_index(drop = True).drop(columns = ['dry_wet_ratio'])
df_dy_wy_ratio_winter_combined_parents  = df_winter_combined_parents[(df_winter_combined_parents['dry_wet_ratio'] > .2) & (df_winter_combined_parents['dry_wet_ratio'] < .6)].reset_index(drop = True).drop(columns = ['dry_wet_ratio'])
df_winter = df_winter.drop(columns = ['dry_wet_ratio'])
df_winter_combined_parents = df_winter_combined_parents.drop(columns = ['dry_wet_ratio'])
print(display(df_winter_combined_parents))

,Field_Name,Crop_Protection_Application_Doses,Soil_Organic_Matter,Yield_kg_per_Ha,Total_N,Total_P,Total_K
0,M6I2035,14.0,2.500000,3049.333333,205.5,48.0,48.0
1,M6I2036,18.0,2.500000,3575.055556,492.0,216.0,216.0
2,M6I2037,13.0,2.500000,3116.200000,198.0,60.0,60.0
3,M6I2009,9.0,2.500000,7678.888889,202.4,95.2,96.0
4,M6I2010,9.0,2.500000,7756.363636,179.4,95.2,90.0
...,...,...,...,...,...,...,...
500,M6I21197,15.0,2.597851,4073.493976,92.0,0.0,0.0
501,M6I21199,17.0,3.120000,6969.677419,184.0,0.0,0.0
502,M6I21200,19.0,2.930000,6969.636364,92.0,0.0,0.0
503,M6I21201,17.0,3.240000,7954.666667,207.0,0.0,0.0


None


In [12]:
df_winter_CO2 = pd.read_csv('/mnt/CO2_prediction/Organized_Work/Datasets/winter_dataset.csv').drop(columns =['Unnamed: 0', 'Yield_kg_per_Ha', 'Yield_CO2_ratio', 'Season'])

df_winter_CO2 = df_winter_CO2[df_winter_CO2['Crop_Protection_Application_Doses'] < 35].reset_index(drop = True)
df_winter_CO2 = df_winter_CO2[(df_winter_CO2['Soil_Organic_Matter'] < 4) & \
                                                       (df_winter_CO2['Soil_Organic_Matter'] > 1)].reset_index(drop = True)
df_winter_CO2 = df_winter_CO2[df_winter_CO2['N_total_kg_per_Ha'] < 400].reset_index(drop = True)
df_winter_CO2 = df_winter_CO2[df_winter_CO2['P_total_kg_per_Ha'] < 200].reset_index(drop = True)
df_winter_CO2 = df_winter_CO2[df_winter_CO2['K_total_kg_per_Ha'] < 200].reset_index(drop = True)

df_winter_CO2 = df_winter_CO2[df_winter_CO2['P_total_kg_per_Ha'] > 0].reset_index(drop = True)
df_winter_CO2 = df_winter_CO2[df_winter_CO2['K_total_kg_per_Ha'] > 0].reset_index(drop = True)
df_winter_CO2 = df_winter_CO2[df_winter_CO2['N_total_kg_per_Ha'] > 0].reset_index(drop = True)

print(display(df_winter_CO2))

df_winter_combined_parents = df_winter_combined_parents[df_winter_combined_parents['Crop_Protection_Application_Doses'] < 35].reset_index(drop = True)
df_winter_combined_parents = df_winter_combined_parents[(df_winter_combined_parents['Soil_Organic_Matter'] < 4) & \
                                                       (df_winter_combined_parents['Soil_Organic_Matter'] > 1)].reset_index(drop = True)
df_winter_combined_parents = df_winter_combined_parents[df_winter_combined_parents['Total_N'] < 400].reset_index(drop = True)
df_winter_combined_parents = df_winter_combined_parents[df_winter_combined_parents['Total_P'] < 200].reset_index(drop = True)
df_winter_combined_parents = df_winter_combined_parents[df_winter_combined_parents['Total_K'] < 200].reset_index(drop = True)

df_winter_combined_parents = df_winter_combined_parents[df_winter_combined_parents['Total_P'] > 0].reset_index(drop = True)
df_winter_combined_parents = df_winter_combined_parents[df_winter_combined_parents['Total_K'] > 0].reset_index(drop = True)
df_winter_combined_parents = df_winter_combined_parents[df_winter_combined_parents['Total_N'] > 0].reset_index(drop = True)

print(display(df_winter_combined_parents))

df_winter_CO2 = df_winter_combined_parents.merge(df_winter_CO2[['Field_Name', 'CO2_kg_per_Ha']], on = 'Field_Name', how = 'left')

print(display(df_winter_CO2))

df_winter_CO2.to_csv('/mnt/CO2_prediction/Organized_Work/Datasets/winter_dataset_filtered_by_outliers.csv')

,Field_Name,CO2_kg_per_Ha,Crop_Protection_Application_Doses,Soil_Organic_Matter,N_total_kg_per_Ha,P_total_kg_per_Ha,K_total_kg_per_Ha
0,M6I2035,-220.35,14.0,2.500000,205.50,48.0,48.0
1,M6I2036,3847.21,18.0,2.500000,246.00,108.0,108.0
2,M6I2037,415.52,13.0,2.500000,198.00,60.0,60.0
3,M6I2009,724.51,9.0,2.500000,201.85,92.6,96.0
4,M6I2010,502.10,9.0,2.500000,178.85,92.6,90.0
...,...,...,...,...,...,...,...
373,M7I21101,373.55,19.0,3.333333,208.10,98.8,90.0
374,M7I21102,470.26,21.0,3.258333,208.10,98.8,90.0
375,M7I21103,784.98,22.0,2.893750,210.40,98.8,90.0
376,M7I21104,327.29,21.0,3.181250,170.30,98.8,90.0


None


,Field_Name,Crop_Protection_Application_Doses,Soil_Organic_Matter,Yield_kg_per_Ha,Total_N,Total_P,Total_K
0,M6I2035,14.0,2.500000,3049.333333,205.5,48.0,48.0
1,M6I2037,13.0,2.500000,3116.200000,198.0,60.0,60.0
2,M6I2009,9.0,2.500000,7678.888889,202.4,95.2,96.0
3,M6I2010,9.0,2.500000,7756.363636,179.4,95.2,90.0
4,M6I2011,10.0,2.500000,8235.200000,179.4,95.2,102.0
...,...,...,...,...,...,...,...
368,M7I21101,19.0,3.333333,5610.500000,208.1,98.8,90.0
369,M7I21102,21.0,3.258333,5610.416667,208.1,98.8,90.0
370,M7I21103,22.0,2.893750,5610.540541,210.4,98.8,90.0
371,M7I21104,21.0,3.181250,8378.648649,170.3,98.8,90.0


None


,Field_Name,Crop_Protection_Application_Doses,Soil_Organic_Matter,Yield_kg_per_Ha,Total_N,Total_P,Total_K,CO2_kg_per_Ha
0,M6I2035,14.0,2.500000,3049.333333,205.5,48.0,48.0,-220.35
1,M6I2037,13.0,2.500000,3116.200000,198.0,60.0,60.0,415.52
2,M6I2009,9.0,2.500000,7678.888889,202.4,95.2,96.0,724.51
3,M6I2010,9.0,2.500000,7756.363636,179.4,95.2,90.0,502.10
4,M6I2011,10.0,2.500000,8235.200000,179.4,95.2,102.0,539.27
...,...,...,...,...,...,...,...,...
368,M7I21101,19.0,3.333333,5610.500000,208.1,98.8,90.0,373.55
369,M7I21102,21.0,3.258333,5610.416667,208.1,98.8,90.0,470.26
370,M7I21103,22.0,2.893750,5610.540541,210.4,98.8,90.0,784.98
371,M7I21104,21.0,3.181250,8378.648649,170.3,98.8,90.0,327.29


None
